<a href="https://colab.research.google.com/github/halfbug/colab/blob/main/organic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install httpx[http2]
!pip install parsel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 931.5 kB/s eta 0:00:00


In [17]:
from collections import defaultdict
from urllib.parse import quote
from httpx import Client
from parsel import Selector

# 1. Create HTTP client with headers that look like a real web browser
client = Client(
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
    },
    follow_redirects=True,
    http2=True,  # use HTTP/2
)


def parse_search_results(selector: Selector):
    """parse search results from google search page"""
    results = []
    for box in selector.xpath("//h1[contains(text(),'Search Results')]/following-sibling::div[1]/div"):
        title = box.xpath(".//h3/text()").get()
        url = box.xpath(".//h3/../@href").get()
        text = "".join(box.xpath(".//div[@data-sncf]//text()").getall())
        if not title or not url:
            continue
        url = url.split("://")[1].replace("www.", "")
        results.append(title, url, text)
    return results


def scrape_search(query: str, page:1):
    """scrape search results for a given keyword"""
    # retrieve the SERP
    url = f"https://www.google.com/search?hl=en&q={quote(query)}" + (f"&start={10*(page-1)}" if page > 1 else "")
    print(f"scraping {query=} {page=}")
    results = defaultdict(list)
    response = client.get(url)
    print(response)
    assert response.status_code == 200, f"failed status_code={response.status_code}"
    # parse SERP for search result data
    selector = Selector(response.text)
    print(selector)
    results["search"].extend(parse_search_results(selector))
    return dict(results)



In [18]:
# example use: scrape 3 pages: 1,2,3
for page in [1, 2, 3]:
    # results = scrape_search("Michael Kors MU281CQ6KW 250")
    results = scrape_search("scrapfly blog", page=page)
    for result in results["search"]:
        print(result)

scraping query='scrapfly blog' page=1
<Response [200 OK]>
<html><body><p>���H�@#t����-����$�5|i�/1�ٕN����Ji�` �'��&lt;�������J�c�س��u�3lw��|�ҁ*N(�T�(��I){�}�7�)��7Mimʖm�c4���VX������FĀ�rP��g�U��W ���lK|_�Ҭ��ఐ_O���bT�1@�W��*(Gn��2;S�Jv����Ox�禾��*7�zSՀ�J�=#[��:-O��G�^y�і:��^��.�R[�/��7?����û���SQ�&amp;ëSݵ�F��lO,�n̚0�z��I�	��cBv� ��ݢK�vP�&gt;�D0�a�����:�9��L��;�2?%���t]�4bW�)�{//B��AJQ�N(�&gt;��o@e������y1Y��E�������A�������56�4�$ɑ�1���m)ײ(|���)��Y�ۉ�T�%�\���z �h��2������}^f.�;����syU���d)hn���,����K�{.�1q���m��7�ۣDŏ۽�&lt;��6�S?B)Ƕ�ܢP�W_Ѝ����&gt;s�L�&gt;~�����fs7�Cی�Q�Z�J�vb"�v�MV�f�o��_g;
�˃g����o�&amp;�:�3�"kM|��,3�oho�;��T�5!��m�H��:����#@�p⹯����ۜ��―��+olM�����kXxN`G��m�p�c_&lt;]�3l�,�-�� v�6Q�K�L���4���u��~���Z�uao������z��z�����)�X+�G�,3��VH�=z���z��]�`�1��Y�M~�v��5���Gץ�r���0���|)/3���sH���_A��~��#:�8
mĲ�����
��4l�̂`�Au^f0:�Y��d7J�Պ��m��J����'1���1[��D
J���͑�����e�Y��R�D�Ѡƹ���P�U蚔�t���)�TIgO�s��^
ٮ���"����&amp;S��s$e|�#��i|�D�&lt;�Ҙm�Nd���}���C�H�(�\��������Q64#O���U"��_��C��іv[#=�

In [22]:
import httpx
import asyncio
from bs4 import BeautifulSoup

In [64]:
async def get_organic_data(query):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4703.0 Safari/537.36"
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(f"https://www.google.com/search?q={query.lower().replace(' ', '+')}&gl=us&hl=en", headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        organic_results = []
        i = 0

        for el in soup.select(".g"):
            print(el.select_one(".VwiC3b"))
            # print(type(el.select_one(".VwiC3b")))
            des = el.select_one(".VwiC3b").text if el.select_one(".VwiC3b") else ""
            title = el.select_one("h3").text if el.select_one("h3") else ""
            link = el.select_one(".yuRUbf   a")["href"] if el.select_one(".yuRUbf   a") else ""
            if title or link or des :
              organic_results.append({
                  "title":title ,
                  "link": link,
                  "description": des,
                  "rank": i+1
              })

              i+=1

        print(organic_results)
        return organic_results


res= await get_organic_data("Apotheke 3 wick candles collection UPC ")

None
None
<div class="VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb" style="-webkit-line-clamp:2"><span>0602573896181. MPN. CHAR-CAN-<em>3Wick</em>. <em>UPC</em>. 602573896181. Brand. <em>APOTHEKE</em>. Scent. Charcoal. Type. Does Not Apply. Category. Home &amp; Garden &gt; <em>Candles</em> &amp; Home ...</span></div>
<div class="VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb" style="-webkit-line-clamp:2"><span>Our <em>candles</em> are hand poured and made with a proprietary blend of essential and perfume-grade oils for a long-lasting fragrance. Each 26 oz <em>candle</em> burns for ...</span></div>
<div class="VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb" style="-webkit-line-clamp:2"><span>EAN: 0602573896181. MPN: CHAR-CAN-<em>3Wick</em>. <em>UPC</em>: 602573896181. Brand: <em>APOTHEKE</em>. Scent: Charcoal. Type: Does Not Apply. <em>Product</em> description. Review rating. 4.9. ( ...</span></div>
<div class="VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb" style="-webkit-line-clamp:2"><span>Made with a

In [65]:
res

[{'title': 'Apotheke 3-Wick Candles Collection Charcoal',
  'link': 'https://www.ebay.com/itm/155928752194',
  'description': '0602573896181. MPN. CHAR-CAN-3Wick. UPC. 602573896181. Brand. APOTHEKE. Scent. Charcoal. Type. Does Not Apply. Category. Home & Garden > Candles & Home\xa0...',
  'rank': 1},
 {'title': 'Apotheke 3-Wick Candles Collection : Health & Household',
  'link': 'https://www.amazon.com/Scented-Candles-Home-Soy-Wax-Jar-Candle/dp/B07PMY9PK6',
  'description': 'Our candles are hand poured and made with a proprietary blend of essential and perfume-grade oils for a long-lasting fragrance. Each 26 oz candle burns for\xa0...',
  'rank': 2},
 {'title': 'discounted purchases Apotheke 3-Wick Candles Collection ...',
  'link': 'https://kannauj-fragrance.com/?p=qk/E512037.html',
  'description': 'EAN: 0602573896181. MPN: CHAR-CAN-3Wick. UPC: 602573896181. Brand: APOTHEKE. Scent: Charcoal. Type: Does Not Apply. Product description. Review rating. 4.9. (\xa0...',
  'rank': 3},
 {'ti